In [1]:
#Getting the train's data and test's data
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize

transform = Compose([
    ToTensor(),
    Normalize((0.5,), (0.5,))
])

data_train = MNIST(root="./", download=True, train=True, transform=transform)
data_test = MNIST(root="./", download=True, train=False, transform=transform)

print(data_train.data.size())
print(data_test.data.size())

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


In [2]:
#Setting a variable to work with GPU
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#Loading data
from torch.utils.data import DataLoader

train_loaded = DataLoader(data_train, batch_size=32, shuffle=True)
test_loaded = DataLoader(data_test, batch_size=32, shuffle=True)

In [40]:
#Creating MNIST model
from torch import nn

class MnistModel(nn.Module):
    def __init__(self):
        super(MnistModel, self).__init__()
        self.cnn_layers = nn.Sequential(
            #CONV1
            nn.Conv2d(1, 6, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            #CONV2
            nn.Conv2d(6, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        
        self.linear_layers = nn.Sequential(
            nn.Linear(32*5*5, 200),
            nn.ReLU(),
            nn.Linear(200, 80),
            nn.ReLU(),
            nn.Linear(80, 10),
        )
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [41]:
#Putting the MnistModel into a variable
model = MnistModel()
model.to(device)

MnistModel(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=800, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=80, bias=True)
    (3): ReLU()
    (4): Linear(in_features=80, out_features=10, bias=True)
  )
)

In [42]:
#Defining an optimizer and loss
from torch import optim

optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [43]:
#Training the model
from tqdm import tqdm
#Defining the numbers of epochs
epochs = 5

#Lists to get all the data about training
train_loss, test_loss = [], []
accuracy_train, accuracy_test = [], []

for epoch in range(epochs):
    total_train_loss = 0
    total_test_loss = 0
    
    model.train()
    
    total = 0
    for index, (image, label) in tqdm(enumerate(train_loaded), desc=f"Fitting Epoch {epoch + 1}"):
        image, label = image.to(device), label.to(device)
        
        optimizer.zero_grad()
        
        pred = model(image)
        
        loss = criterion(pred, label)
        total_train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
        pred = nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
    
    train_accuracy = total / len(data_train)
    total_train_loss = total_train_loss / (index + 1)
    
    accuracy_train.append(train_accuracy)
    train_loss.append(total_train_loss)
    
    #Validating the model
    model.eval()
    total = 0
    for index, (image, label) in tqdm(enumerate(test_loaded), desc="Validating the model"):
        image, label = image.to(device), label.to(device)
        pred = model(image)
        
        loss = criterion(pred, label)
        total_test_loss += loss.item()
        
        pred = nn.functional.softmax(pred, dim=1)
        for i, p in enumerate(pred):
            if label[i] == torch.max(p.data, 0)[1]:
                total = total + 1
    test_accuracy = total / len(data_test)
    total_test_loss = total_test_loss / (index + 1)
    
    accuracy_test.append(test_accuracy)
    test_loss.append(total_test_loss)
    
    print("Epoch: {}/{}  ".format(epoch + 1, epochs),
            "Training loss: {:.4f}  ".format(total_train_loss),
            "Testing loss: {:.4f}  ".format(total_test_loss),
            "Train accuracy: {:.4f}  ".format(train_accuracy),
            "Test accuracy: {:.4f}  ".format(test_accuracy))

Fitting Epoch 1: 1875it [00:20, 92.21it/s]
Validating the model: 313it [00:02, 105.94it/s]


Epoch: 1/5   Training loss: 0.1620   Testing loss: 0.0805   Train accuracy: 0.9537   Test accuracy: 0.9780  


Fitting Epoch 2: 1875it [00:20, 92.33it/s]
Validating the model: 313it [00:03, 90.79it/s]


Epoch: 2/5   Training loss: 0.1062   Testing loss: 0.1104   Train accuracy: 0.9731   Test accuracy: 0.9703  


Fitting Epoch 3: 1875it [00:22, 82.26it/s]
Validating the model: 313it [00:02, 105.21it/s]


Epoch: 3/5   Training loss: 0.1012   Testing loss: 0.1071   Train accuracy: 0.9758   Test accuracy: 0.9749  


Fitting Epoch 4: 1875it [00:21, 87.45it/s]
Validating the model: 313it [00:03, 84.57it/s]


Epoch: 4/5   Training loss: 0.0916   Testing loss: 0.1021   Train accuracy: 0.9796   Test accuracy: 0.9794  


Fitting Epoch 5: 1875it [00:26, 71.78it/s]
Validating the model: 313it [00:03, 90.68it/s]

Epoch: 5/5   Training loss: 0.0934   Testing loss: 0.0689   Train accuracy: 0.9800   Test accuracy: 0.9841  


In [44]:
#Saving the model and the model parameters
torch.save(model.state_dict(),"../Pytorch/Model/model_dict")
torch.save(model, "../Pytorch/Model/model.pt")
print("Model Saved")

Model Saved
